In [1]:
import argparse
import torch
import datetime
import json
import yaml
import os

from dataset_pm25 import get_dataloader
from main_model import CSDI_PM25,CSDI_Physio
from utils import train, evaluate

In [2]:
parser = argparse.ArgumentParser(description="CSDI")
# parser.add_argument("--config", type=str, default="base.yaml")
parser.add_argument("--config", type=str, default="small.yaml")

parser.add_argument('--device', default='cuda:0', help='Device for Attack')
parser.add_argument("--seed", type=int, default=1)
parser.add_argument("--testmissingratio", type=float, default=0.1)
parser.add_argument(
    "--nfold", type=int, default=0, help="for 5fold test (valid value:[0-4])"
)
parser.add_argument("--unconditional", action="store_true")
parser.add_argument("--modelfolder", type=str, default="")
parser.add_argument("--nsample", type=int, default=100)

args = parser.parse_args()
print(args)

path = "config/" + args.config
with open(path, "r") as f:
    config = yaml.safe_load(f)

config["model"]["is_unconditional"] = args.unconditional
config["model"]["test_missing_ratio"] = args.testmissingratio

print(json.dumps(config, indent=4))

current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
foldername = "./save/physio_fold" + str(args.nfold) + "_" + current_time + "/"
print('model folder:', foldername)
os.makedirs(foldername, exist_ok=True)
with open(foldername + "config.json", "w") as f:
    json.dump(config, f, indent=4)

usage: ipykernel_launcher.py [-h] [--config CONFIG] [--device DEVICE]
                             [--seed SEED]
                             [--testmissingratio TESTMISSINGRATIO]
                             [--nfold NFOLD] [--unconditional]
                             [--modelfolder MODELFOLDER] [--nsample NSAMPLE]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"04a46126-126b-4aaf-b6b6-589f8ba843cc" --shell=9002 --transport="tcp" --iopub=9004 --f=/tmp/tmp-25403dWBh23mIUedW.json


SystemExit: 2

/home/zenus/anaconda3/envs/pytorch_1.10.1/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:

path = "config/" + args.config
with open(path, "r") as f:
    config = yaml.safe_load(f)

config["model"]["is_unconditional"] = args.unconditional
config["model"]["test_missing_ratio"] = args.testmissingratio

print(json.dumps(config, indent=4))

current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
foldername = "./save/physio_fold" + str(args.nfold) + "_" + current_time + "/"
print('model folder:', foldername)
os.makedirs(foldername, exist_ok=True)
with open(foldername + "config.json", "w") as f:
    json.dump(config, f, indent=4)

AttributeError: 'Namespace' object has no attribute 'testmissingratio'

In [5]:
train_loader, valid_loader, test_loader, scaler, mean_scaler = get_dataloader(
    config["train"]["batch_size"], device=args.device, validindex=args.validationindex
)
model = CSDI_PM25(config, args.device).to(args.device)

In [17]:
item = next(iter(valid_loader))
item["observed_data"].shape

torch.Size([16, 36, 36])

In [18]:
train_loader, valid_loader, test_loader, scaler, mean_scaler = get_dataloader(
    config["train"]["batch_size"], device=args.device, validindex=args.validationindex
)
model = CSDI_PM25(config, args.device).to(args.device)